In [10]:
%pip install xmltodict
import xmltodict

# Read and parse the XML file
with open("2025-12326.xml", "r", encoding="utf-8", errors="replace") as file:
    xml_content = file.read()

policy_dict = xmltodict.parse(xml_content)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Refined extraction logic for regulatory requirements and metadata from NEPA XML
import re
import json

# Helper function to identify requirement-like sentences
requirement_keywords = [
    'must', 'shall', 'required', 'mandate', 'prohibit', 'forbid', 'ensure', 'direct', 'provision', 'deadline', 'limit', 'review', 'prepare', 'submit', 'report', 'determine', 'specify', 'clarify', 'add', 'remove', 'revise', 'publish', 'provide', 'consider', 'apply', 'adopt', 'implement', 'establish', 'define', 'categorical exclusion', 'environmental assessment', 'environmental impact statement'
]

# Flatten paragraphs from SUPLINF
suplinf = policy_dict['RULE']['SUPLINF']
paragraphs = []
for p in suplinf.get('P', []):
    if isinstance(p, str) and p.strip():
        paragraphs.append(p.strip())

# Extract requirements and decision points
extracted_rules = []
for idx, para in enumerate(paragraphs):
    # Check for requirement keywords
    if any(kw in para.lower() for kw in requirement_keywords):
        # Try to extract citation and effective date from context
        citation_match = re.search(r'(\d+\s*U\.S\.C\.\s*\d+[a-zA-Z0-9\(\)]*)', para)
        citation = citation_match.group(0) if citation_match else None
        # Use known effective date from context if not found
        effective_date = '2025-04-11'
        # Build rule
        rule = {
            "id": f"rule_{idx+1}",
            "text": para,
            "logic": "To be refined (AI-assisted translation)",
            "metadata": {
                "citation": citation,
                "effective_date": effective_date
            }
        }
        extracted_rules.append(rule)

# Display the extracted rules as JSON
print(json.dumps(extracted_rules, indent=2))

# Summary: This cell refines extraction by scanning for regulatory keywords and attempts to capture citations and effective dates for each rule. You can further refine logic or prompt AI to translate 'text' into executable code logic.

NameError: name 'json' is not defined